<a href="https://colab.research.google.com/github/YasiruMethmal123/See-People/blob/master/SeePeopleAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openpyxl sentence-transformers scikit-learn numpy -q

import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import pickle
from google.colab import files
import warnings
warnings.filterwarnings('ignore')